# ORPFlow - Training Pipeline (GPU A100)

**IMPORTANTE: Este notebook implementa um pipeline LIVRE de Data Leakage**

## Garantias Anti-Leakage:
1. Split temporal POR SÍMBOLO (elimina cross-symbol contamination)
2. Scaler fit APENAS nos dados de treino
3. Features calculadas usando apenas dados passados (rolling windows backward-looking)
4. Cada modelo treinado de forma independente
5. Sem vazamento de informação entre train/val/test
6. Sem bfill() que causaria leakage
7. RL treinado APENAS com dados de treino

## Correção v2.0:
- **ANTES**: Global sort + index split → cross-symbol contamination nas bordas
- **AGORA**: Split por símbolo INDEPENDENTE → cada símbolo tem seu próprio split temporal

## Modelos:
- ML: LightGBM, XGBoost
- DL: LSTM, CNN  
- RL: D4PG+EVT, MARL

## 1. Setup do Ambiente

In [11]:
# Verificar GPU
!nvidia-smi

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memória GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

Sun Dec 21 02:49:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   35C    P0             61W /  400W |   25621MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [12]:
# Instalar dependências
!pip install -q lightgbm xgboost torch torchvision --upgrade
!pip install -q onnx onnxruntime-gpu onnxmltools skl2onnx
!pip install -q pandas numpy scikit-learn scipy
!pip install -q aiohttp tqdm pyyaml matplotlib seaborn
!pip install -q python-binance

print("Dependências instaladas!")

Dependências instaladas!


In [13]:
# Imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import lightgbm as lgb
import xgboost as xgb
from pathlib import Path
import json
import time
import logging
import warnings
warnings.filterwarnings('ignore')

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Diretórios
DATA_DIR = Path("data")
TRAINED_DIR = Path("trained")
ONNX_DIR = TRAINED_DIR / "onnx"

DATA_DIR.mkdir(parents=True, exist_ok=True)
TRAINED_DIR.mkdir(parents=True, exist_ok=True)
ONNX_DIR.mkdir(parents=True, exist_ok=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {DEVICE}")

Device: cuda


## 2. Coleta de Dados

In [14]:
import aiohttp
import asyncio
from datetime import datetime, timedelta
from tqdm.notebook import tqdm

async def fetch_klines(symbol: str, interval: str = "1m", days: int = 90) -> pd.DataFrame:
    """Coleta dados históricos da Binance"""
    base_url = "https://api.binance.com/api/v3/klines"
    end_time = datetime.utcnow()
    start_time = end_time - timedelta(days=days)

    all_data = []
    current = start_time

    async with aiohttp.ClientSession() as session:
        while current < end_time:
            params = {
                "symbol": symbol,
                "interval": interval,
                "startTime": int(current.timestamp() * 1000),
                "endTime": int(min(current + timedelta(days=1), end_time).timestamp() * 1000),
                "limit": 1440
            }

            async with session.get(base_url, params=params) as resp:
                data = await resp.json()
                all_data.extend(data)

            current += timedelta(days=1)
            await asyncio.sleep(0.1)  # Rate limit

    columns = ["open_time", "open", "high", "low", "close", "volume",
               "close_time", "quote_volume", "trades", "taker_buy_base",
               "taker_buy_quote", "ignore"]

    df = pd.DataFrame(all_data, columns=columns)
    df["open_time"] = pd.to_datetime(df["open_time"], unit="ms")
    df["close_time"] = pd.to_datetime(df["close_time"], unit="ms")

    for col in ["open", "high", "low", "close", "volume", "quote_volume",
                "taker_buy_base", "taker_buy_quote"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    df["symbol"] = symbol
    return df.drop_duplicates(subset=["open_time"]).sort_values("open_time")

In [15]:
# Coletar dados
SYMBOLS = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT"]
DAYS = 90

all_data = []
for symbol in tqdm(SYMBOLS, desc="Coletando dados"):
    df = await fetch_klines(symbol, days=DAYS)
    all_data.append(df)
    print(f"{symbol}: {len(df)} rows")

raw_data = pd.concat(all_data, ignore_index=True)
print(f"\nTotal: {len(raw_data)} rows")

Coletando dados:   0%|          | 0/4 [00:00<?, ?it/s]

BTCUSDT: 90000 rows
ETHUSDT: 90000 rows
BNBUSDT: 90000 rows
SOLUSDT: 90000 rows

Total: 360000 rows


## 3. Feature Engineering (Sem Leakage)

**IMPORTANTE:** Todas as features usam APENAS dados passados (rolling windows olham para trás)

In [16]:
def calculate_features(df: pd.DataFrame, windows: list = [5, 10, 20, 50, 100]) -> pd.DataFrame:
    """
    Calcula features usando APENAS dados passados.
    Todas as rolling windows olham para trás, sem leakage.
    """
    df = df.copy()

    # Returns (olham para o passado)
    df["return_1"] = df["close"].pct_change()
    df["log_return"] = np.log(df["close"] / df["close"].shift(1))

    for w in windows:
        df[f"return_{w}"] = df["close"].pct_change(w)
        df[f"log_return_{w}"] = np.log(df["close"] / df["close"].shift(w))

    # Volatility (rolling olha para trás)
    for w in windows:
        df[f"volatility_{w}"] = df["log_return"].rolling(window=w).std() * np.sqrt(252 * 24 * 60)

        # Parkinson volatility
        df[f"parkinson_vol_{w}"] = np.sqrt(
            (1 / (4 * np.log(2))) *
            ((np.log(df["high"] / df["low"]) ** 2).rolling(window=w).mean())
        ) * np.sqrt(252 * 24 * 60)

        # Garman-Klass volatility
        log_hl = np.log(df["high"] / df["low"]) ** 2
        log_co = np.log(df["close"] / df["open"]) ** 2
        df[f"gk_vol_{w}"] = np.sqrt(
            (0.5 * log_hl - (2 * np.log(2) - 1) * log_co).rolling(window=w).mean()
        ) * np.sqrt(252 * 24 * 60)

    # Momentum (olham para o passado)
    for w in windows:
        df[f"momentum_{w}"] = df["close"] / df["close"].shift(w) - 1
        df[f"roc_{w}"] = (df["close"] - df["close"].shift(w)) / df["close"].shift(w) * 100
        df[f"ma_{w}"] = df["close"].rolling(window=w).mean()
        df[f"ma_cross_{w}"] = (df["close"] - df[f"ma_{w}"]) / df[f"ma_{w}"]

    # RSI (olha para o passado)
    for w in [14, 21]:
        delta = df["close"].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=w).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=w).mean()
        rs = gain / loss
        df[f"rsi_{w}"] = 100 - (100 / (1 + rs))

    # Order flow (olham para o passado)
    df["spread_proxy"] = (df["high"] - df["low"]) / df["close"] * 10000
    df["volume_imbalance"] = (df["taker_buy_base"] - (df["volume"] - df["taker_buy_base"])) / df["volume"]
    df["ofi"] = df["taker_buy_base"] / df["volume"]

    for w in windows:
        df[f"ofi_ma_{w}"] = df["ofi"].rolling(window=w).mean()
        df[f"ofi_std_{w}"] = df["ofi"].rolling(window=w).std()
        df[f"volume_ma_{w}"] = df["volume"].rolling(window=w).mean()
        df[f"volume_std_{w}"] = df["volume"].rolling(window=w).std()
        df[f"trades_ma_{w}"] = df["trades"].rolling(window=w).mean()

    # Microstructure (olham para o passado)
    df["amihud"] = np.abs(df["log_return"]) / df["quote_volume"]
    for w in windows:
        df[f"amihud_ma_{w}"] = df["amihud"].rolling(window=w).mean()
        df[f"kyle_lambda_{w}"] = df["log_return"].rolling(window=w).std() / df["volume"].rolling(window=w).mean()

    # Time features (sem leakage - são características do momento)
    df["hour"] = df["open_time"].dt.hour
    df["day_of_week"] = df["open_time"].dt.dayofweek
    df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
    df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)
    df["dow_sin"] = np.sin(2 * np.pi * df["day_of_week"] / 7)
    df["dow_cos"] = np.cos(2 * np.pi * df["day_of_week"] / 7)

    return df


def calculate_targets(df: pd.DataFrame, horizons: list = [1, 5, 15, 30]) -> pd.DataFrame:
    """
    Calcula targets (valores FUTUROS que queremos prever).
    Usamos shift(-h) para pegar retorno futuro.
    """
    df = df.copy()

    for h in horizons:
        # Retorno futuro (o que queremos prever)
        df[f"target_return_{h}"] = df["close"].shift(-h) / df["close"] - 1
        # Direção futura
        df[f"target_direction_{h}"] = (df[f"target_return_{h}"] > 0).astype(int)

    return df

In [17]:
def get_feature_columns(df: pd.DataFrame) -> list:
    """Retorna apenas colunas de features (exclui targets e metadata)"""
    exclude_prefixes = ["target_", "open_time", "close_time", "symbol", "ignore"]
    exclude_cols = ["open", "high", "low", "close", "volume", "quote_volume",
                    "trades", "taker_buy_base", "taker_buy_quote", "hour", "day_of_week"]

    feature_cols = []
    for col in df.columns:
        if any(col.startswith(p) for p in exclude_prefixes):
            continue
        if col in exclude_cols:
            continue
        feature_cols.append(col)

    return feature_cols

In [18]:
# Processar cada símbolo e ARMAZENAR SEPARADAMENTE
# NÃO concatenar aqui - faremos split por símbolo!
processed_by_symbol = {}

for symbol in raw_data["symbol"].unique():
    symbol_df = raw_data[raw_data["symbol"] == symbol].copy()
    symbol_df = symbol_df.sort_values("open_time").reset_index(drop=True)

    print(f"Processando {symbol}: {len(symbol_df)} rows")

    # Calcular features (olham apenas para o passado)
    symbol_df = calculate_features(symbol_df)

    # Calcular targets (valores futuros)
    symbol_df = calculate_targets(symbol_df)

    # Substituir infinitos por NaN
    symbol_df = symbol_df.replace([np.inf, -np.inf], np.nan)

    # Dropar warmup period (primeiras 100 rows onde features são NaN)
    # NÃO usamos bfill() que causaria leakage!
    warmup = 100
    symbol_df = symbol_df.iloc[warmup:].reset_index(drop=True)

    # Dropar rows onde target é NaN (final dos dados)
    target_cols = [c for c in symbol_df.columns if c.startswith("target_")]
    symbol_df = symbol_df.dropna(subset=target_cols)

    # Dropar qualquer NaN restante nas features
    feature_cols = get_feature_columns(symbol_df)
    symbol_df = symbol_df.dropna(subset=feature_cols)

    # Armazenar por símbolo para split independente
    processed_by_symbol[symbol] = symbol_df
    print(f"  -> {len(symbol_df)} rows após processamento")

total_rows = sum(len(df) for df in processed_by_symbol.values())
print(f"\nTotal processado: {total_rows} rows")
print(f"Símbolos: {list(processed_by_symbol.keys())}")

Processando BTCUSDT: 90000 rows
  -> 89870 rows após processamento
Processando ETHUSDT: 90000 rows
  -> 89870 rows após processamento
Processando BNBUSDT: 90000 rows
  -> 89870 rows após processamento
Processando SOLUSDT: 90000 rows
  -> 89870 rows após processamento

Total processado: 359480 rows
Símbolos: ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'SOLUSDT']


## 4. Split Temporal POR SÍMBOLO (CORREÇÃO CRÍTICA!)

**PROBLEMA ANTERIOR:**
- Global sort por `open_time` + split por índice
- No boundary train/val, diferentes símbolos no MESMO timestamp ficavam em splits diferentes
- Exemplo: BTCUSDT às 10:00 no TRAIN, ETHUSDT às 10:00 no VAL → **LEAKAGE!**

**SOLUÇÃO:**
- Split CADA símbolo independentemente (70/15/15)
- Depois concatena train de todos símbolos, val de todos, test de todos
- Garante que para CADA símbolo, train < val < test temporalmente
- Elimina cross-symbol contamination completamente!

In [19]:
# Configurações
TARGET_COL = "target_return_5"
SEQUENCE_LENGTH = 60
TEST_SIZE = 0.15
VAL_SIZE = 0.15

# Pegar feature columns de qualquer símbolo (são as mesmas para todos)
sample_symbol = list(processed_by_symbol.keys())[0]
feature_cols = get_feature_columns(processed_by_symbol[sample_symbol])
print(f"Features: {len(feature_cols)}")
print(f"Target: {TARGET_COL}")

Features: 92
Target: target_return_5


In [20]:
# CORREÇÃO CRÍTICA: Split POR SÍMBOLO para evitar cross-symbol contamination!
# Cada símbolo é dividido independentemente, depois concatenamos

train_dfs = []
val_dfs = []
test_dfs = []

print("Split temporal POR SÍMBOLO (elimina cross-symbol leakage):\n")

for symbol, symbol_df in processed_by_symbol.items():
    n = len(symbol_df)
    train_end = int(n * (1 - TEST_SIZE - VAL_SIZE))
    val_end = int(n * (1 - TEST_SIZE))

    symbol_train = symbol_df.iloc[:train_end].copy()
    symbol_val = symbol_df.iloc[train_end:val_end].copy()
    symbol_test = symbol_df.iloc[val_end:].copy()

    train_dfs.append(symbol_train)
    val_dfs.append(symbol_val)
    test_dfs.append(symbol_test)

    print(f"{symbol}:")
    print(f"  Train: {len(symbol_train)} ({symbol_train['open_time'].min()} to {symbol_train['open_time'].max()})")
    print(f"  Val:   {len(symbol_val)} ({symbol_val['open_time'].min()} to {symbol_val['open_time'].max()})")
    print(f"  Test:  {len(symbol_test)} ({symbol_test['open_time'].min()} to {symbol_test['open_time'].max()})")
    print()

# Concatenar os splits (agora CADA split tem dados do mesmo período temporal!)
train_df = pd.concat(train_dfs, ignore_index=True).sort_values("open_time").reset_index(drop=True)
val_df = pd.concat(val_dfs, ignore_index=True).sort_values("open_time").reset_index(drop=True)
test_df = pd.concat(test_dfs, ignore_index=True).sort_values("open_time").reset_index(drop=True)

total = len(train_df) + len(val_df) + len(test_df)
print("=" * 60)
print("RESUMO DO SPLIT (SEM CROSS-SYMBOL CONTAMINATION):")
print("=" * 60)
print(f"Train: {len(train_df)} ({len(train_df)/total:.1%})")
print(f"Val:   {len(val_df)} ({len(val_df)/total:.1%})")
print(f"Test:  {len(test_df)} ({len(test_df)/total:.1%})")

# Verificar separação temporal REAL
print(f"\nTrain period: {train_df['open_time'].min()} to {train_df['open_time'].max()}")
print(f"Val period:   {val_df['open_time'].min()} to {val_df['open_time'].max()}")
print(f"Test period:  {test_df['open_time'].min()} to {test_df['open_time'].max()}")

# VALIDAÇÃO: Garantir que não há overlap!
train_max = train_df['open_time'].max()
val_min = val_df['open_time'].min()
val_max = val_df['open_time'].max()
test_min = test_df['open_time'].min()

if train_max < val_min and val_max < test_min:
    print("\n✅ VALIDAÇÃO: Sem overlap temporal entre splits!")
else:
    print("\n⚠️ AVISO: Ainda pode haver overlap em bordas (mas sem cross-symbol contamination)")

Split temporal POR SÍMBOLO (elimina cross-symbol leakage):

BTCUSDT:
  Train: 62908 (2025-09-22 04:31:00 to 2025-11-24 02:58:00)
  Val:   13481 (2025-11-24 02:59:00 to 2025-12-07 10:59:00)
  Test:  13481 (2025-12-07 11:00:00 to 2025-12-20 19:00:00)

ETHUSDT:
  Train: 62908 (2025-09-22 04:31:00 to 2025-11-24 02:58:00)
  Val:   13481 (2025-11-24 02:59:00 to 2025-12-07 10:59:00)
  Test:  13481 (2025-12-07 11:00:00 to 2025-12-20 19:00:00)

BNBUSDT:
  Train: 62908 (2025-09-22 04:32:00 to 2025-11-24 02:59:00)
  Val:   13481 (2025-11-24 03:00:00 to 2025-12-07 11:00:00)
  Test:  13481 (2025-12-07 11:01:00 to 2025-12-20 19:01:00)

SOLUSDT:
  Train: 62908 (2025-09-22 04:32:00 to 2025-11-24 02:59:00)
  Val:   13481 (2025-11-24 03:00:00 to 2025-12-07 11:00:00)
  Test:  13481 (2025-12-07 11:01:00 to 2025-12-20 19:01:00)

RESUMO DO SPLIT (SEM CROSS-SYMBOL CONTAMINATION):
Train: 251632 (70.0%)
Val:   53924 (15.0%)
Test:  53924 (15.0%)

Train period: 2025-09-22 04:31:00 to 2025-11-24 02:59:00
Val peri

In [21]:
# Extrair X e y ANTES do scaling
X_train_raw = train_df[feature_cols].values
y_train = train_df[TARGET_COL].values

X_val_raw = val_df[feature_cols].values
y_val = val_df[TARGET_COL].values

X_test_raw = test_df[feature_cols].values
y_test = test_df[TARGET_COL].values

print(f"X_train shape: {X_train_raw.shape}")
print(f"X_val shape: {X_val_raw.shape}")
print(f"X_test shape: {X_test_raw.shape}")

X_train shape: (251632, 92)
X_val shape: (53924, 92)
X_test shape: (53924, 92)


## 5. Scaling (Fit APENAS no Treino)

**CRÍTICO:** Scaler é fitado APENAS nos dados de treino!

In [22]:
# Scaler para ML - fit APENAS no treino!
scaler_ml = RobustScaler()

X_train_ml = scaler_ml.fit_transform(X_train_raw)  # FIT apenas aqui
X_val_ml = scaler_ml.transform(X_val_raw)          # TRANSFORM apenas
X_test_ml = scaler_ml.transform(X_test_raw)        # TRANSFORM apenas

print("ML Data (scaled):")
print(f"  Train: {X_train_ml.shape}")
print(f"  Val: {X_val_ml.shape}")
print(f"  Test: {X_test_ml.shape}")

ML Data (scaled):
  Train: (251632, 92)
  Val: (53924, 92)
  Test: (53924, 92)


In [23]:
# Scaler SEPARADO para DL - fit APENAS no treino!
scaler_dl = RobustScaler()

X_train_scaled_dl = scaler_dl.fit_transform(X_train_raw)  # FIT apenas aqui
X_val_scaled_dl = scaler_dl.transform(X_val_raw)          # TRANSFORM apenas
X_test_scaled_dl = scaler_dl.transform(X_test_raw)        # TRANSFORM apenas

def create_sequences(X, y, seq_length):
    """Cria sequências para LSTM/CNN"""
    X_seq, y_seq = [], []
    for i in range(seq_length, len(X)):
        X_seq.append(X[i-seq_length:i])
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)

# Criar sequências APÓS o scaling
X_train_seq, y_train_seq = create_sequences(X_train_scaled_dl, y_train, SEQUENCE_LENGTH)
X_val_seq, y_val_seq = create_sequences(X_val_scaled_dl, y_val, SEQUENCE_LENGTH)
X_test_seq, y_test_seq = create_sequences(X_test_scaled_dl, y_test, SEQUENCE_LENGTH)

print("\nDL Data (sequences):")
print(f"  Train: {X_train_seq.shape}")
print(f"  Val: {X_val_seq.shape}")
print(f"  Test: {X_test_seq.shape}")


DL Data (sequences):
  Train: (251572, 60, 92)
  Val: (53864, 60, 92)
  Test: (53864, 60, 92)


In [24]:
# Dados para RL - usar APENAS dados de treino para inicialização
# RL aprende online, então usamos dados de treino para o ambiente
ohlcv_cols = ["open", "high", "low", "close", "volume"]

# Scaler SEPARADO para RL - fit APENAS no treino!
scaler_rl = RobustScaler()

rl_train_data = train_df[ohlcv_cols].values
rl_train_features = scaler_rl.fit_transform(train_df[feature_cols].values)  # FIT apenas aqui

# Para avaliação final do RL
rl_test_data = test_df[ohlcv_cols].values
rl_test_features = scaler_rl.transform(test_df[feature_cols].values)  # TRANSFORM apenas

print("\nRL Data:")
print(f"  Train OHLCV: {rl_train_data.shape}")
print(f"  Train Features: {rl_train_features.shape}")


RL Data:
  Train OHLCV: (251632, 5)
  Train Features: (251632, 92)


In [25]:
# Armazenar resultados
all_metrics = {}
trained_models = {}

## 6. Funções de Avaliação

In [26]:
def calculate_trading_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> dict:
    """Calcula métricas de trading"""
    # Acurácia direcional
    direction_accuracy = np.mean(np.sign(y_true) == np.sign(y_pred))

    # Retornos da estratégia (trade na direção da previsão)
    strategy_returns = y_true * np.sign(y_pred)

    # Sharpe ratio (anualizado)
    sharpe_ratio = (
        np.mean(strategy_returns) / (np.std(strategy_returns) + 1e-8)
    ) * np.sqrt(252 * 24 * 60)

    # Sortino ratio
    downside = strategy_returns[strategy_returns < 0]
    sortino_ratio = (
        np.mean(strategy_returns) / (np.std(downside) + 1e-8)
    ) * np.sqrt(252 * 24 * 60) if len(downside) > 0 else 0

    # Win rate
    win_rate = np.mean(strategy_returns > 0)

    # Profit factor
    gains = strategy_returns[strategy_returns > 0].sum()
    losses = np.abs(strategy_returns[strategy_returns < 0].sum())
    profit_factor = gains / (losses + 1e-8)

    # Max drawdown
    cumulative = np.cumsum(strategy_returns)
    running_max = np.maximum.accumulate(cumulative)
    max_drawdown = np.max(running_max - cumulative)

    return {
        "direction_accuracy": direction_accuracy,
        "sharpe_ratio": sharpe_ratio,
        "sortino_ratio": sortino_ratio,
        "win_rate": win_rate,
        "profit_factor": profit_factor,
        "max_drawdown": max_drawdown,
        "total_return": cumulative[-1] if len(cumulative) > 0 else 0
    }


def evaluate_model(y_true: np.ndarray, y_pred: np.ndarray, model_name: str) -> dict:
    """Avalia modelo com todas as métricas"""
    metrics = {
        "mse": mean_squared_error(y_true, y_pred),
        "mae": mean_absolute_error(y_true, y_pred),
        "rmse": np.sqrt(mean_squared_error(y_true, y_pred)),
        "r2": r2_score(y_true, y_pred)
    }

    trading_metrics = calculate_trading_metrics(y_true, y_pred)
    metrics.update(trading_metrics)

    print(f"\n{model_name} - Test Metrics:")
    print(f"  MSE: {metrics['mse']:.6f}")
    print(f"  Sharpe: {metrics['sharpe_ratio']:.4f}")
    print(f"  Win Rate: {metrics['win_rate']:.2%}")
    print(f"  Profit Factor: {metrics['profit_factor']:.2f}")

    return metrics

## 7. Treinamento: LightGBM

In [27]:
print("="*60)
print("TREINANDO LIGHTGBM")
print("="*60)

lgb_params = {
    "objective": "regression",
    "metric": "mse",
    "boosting_type": "gbdt",
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "verbose": -1,
}

train_data = lgb.Dataset(X_train_ml, label=y_train, feature_name=feature_cols)
val_data = lgb.Dataset(X_val_ml, label=y_val, feature_name=feature_cols, reference=train_data)

start_time = time.time()
lgb_model = lgb.train(
    lgb_params,
    train_data,
    num_boost_round=2000,
    valid_sets=[train_data, val_data],
    valid_names=["train", "val"],
    callbacks=[
        lgb.early_stopping(stopping_rounds=100),
        lgb.log_evaluation(period=200)
    ]
)
lgb_train_time = time.time() - start_time

# Avaliar no TEST set
y_pred_lgb = lgb_model.predict(X_test_ml)
lgb_metrics = evaluate_model(y_test, y_pred_lgb, "LightGBM")
lgb_metrics["train_time"] = lgb_train_time

all_metrics["lightgbm"] = lgb_metrics
trained_models["lightgbm"] = lgb_model

# Salvar
lgb_model.save_model(str(TRAINED_DIR / "lightgbm_model.lgb"))
print(f"\nTempo de treino: {lgb_train_time:.1f}s")

TREINANDO LIGHTGBM
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[3]	train's l2: 6.22672e-06	val's l2: 5.8122e-06

LightGBM - Test Metrics:
  MSE: 0.000005
  Sharpe: 0.5746
  Win Rate: 49.55%
  Profit Factor: 1.00

Tempo de treino: 2.5s


## 8. Treinamento: XGBoost (GPU)

In [28]:
print("="*60)
print("TREINANDO XGBOOST (GPU)")
print("="*60)

xgb_params = {
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "max_depth": 8,
    "learning_rate": 0.05,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "tree_method": "hist",
    "device": "cuda",  # GPU A100
}

dtrain = xgb.DMatrix(X_train_ml, label=y_train, feature_names=feature_cols)
dval = xgb.DMatrix(X_val_ml, label=y_val, feature_names=feature_cols)
dtest = xgb.DMatrix(X_test_ml, feature_names=feature_cols)

start_time = time.time()
xgb_model = xgb.train(
    xgb_params,
    dtrain,
    num_boost_round=2000,
    evals=[(dtrain, "train"), (dval, "val")],
    early_stopping_rounds=100,
    verbose_eval=200
)
xgb_train_time = time.time() - start_time

# Avaliar no TEST set
y_pred_xgb = xgb_model.predict(dtest)
xgb_metrics = evaluate_model(y_test, y_pred_xgb, "XGBoost")
xgb_metrics["train_time"] = xgb_train_time

all_metrics["xgboost"] = xgb_metrics
trained_models["xgboost"] = xgb_model

# Salvar
xgb_model.save_model(str(TRAINED_DIR / "xgboost_model.json"))
print(f"\nTempo de treino: {xgb_train_time:.1f}s")

TREINANDO XGBOOST (GPU)
[0]	train-rmse:0.00250	val-rmse:0.00241
[100]	train-rmse:0.00216	val-rmse:0.00288

XGBoost - Test Metrics:
  MSE: 0.000011
  Sharpe: 1.6978
  Win Rate: 49.27%
  Profit Factor: 1.01

Tempo de treino: 0.9s


## 9. Treinamento: LSTM

In [29]:
class LSTMNetwork(nn.Module):
    def __init__(self, input_size, hidden_size=256, num_layers=3, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.attention = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1)
        )
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 2, 1)
        )

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        attn_weights = torch.softmax(self.attention(lstm_out), dim=1)
        context = torch.sum(attn_weights * lstm_out, dim=1)
        return self.fc(context).squeeze(-1)

In [30]:
print("="*60)
print("TREINANDO LSTM (CUDA)")
print("="*60)

num_features = X_train_seq.shape[2]
lstm_model = LSTMNetwork(input_size=num_features, hidden_size=256, num_layers=3).to(DEVICE)

optimizer = torch.optim.AdamW(lstm_model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=5)
criterion = nn.MSELoss()

train_loader = DataLoader(
    TensorDataset(torch.FloatTensor(X_train_seq), torch.FloatTensor(y_train_seq)),
    batch_size=256, shuffle=True
)
val_loader = DataLoader(
    TensorDataset(torch.FloatTensor(X_val_seq), torch.FloatTensor(y_val_seq)),
    batch_size=256, shuffle=False
)

best_val_loss = float("inf")
patience_counter = 0
patience = 15
epochs = 150

start_time = time.time()
for epoch in range(epochs):
    lstm_model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
        optimizer.zero_grad()
        output = lstm_model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(lstm_model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)

    lstm_model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            output = lstm_model(X_batch)
            val_loss += criterion(output, y_batch).item()
    val_loss /= len(val_loader)

    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_state = lstm_model.state_dict().copy()
    else:
        patience_counter += 1

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        break

lstm_model.load_state_dict(best_state)
lstm_train_time = time.time() - start_time

# Avaliar no TEST set
lstm_model.eval()
with torch.no_grad():
    y_pred_lstm = lstm_model(torch.FloatTensor(X_test_seq).to(DEVICE)).cpu().numpy()

lstm_metrics = evaluate_model(y_test_seq, y_pred_lstm, "LSTM")
lstm_metrics["train_time"] = lstm_train_time

all_metrics["lstm"] = lstm_metrics
trained_models["lstm"] = lstm_model

# Salvar
torch.save(lstm_model.state_dict(), str(TRAINED_DIR / "lstm_model.pt"))
print(f"\nTempo de treino: {lstm_train_time:.1f}s")

TREINANDO LSTM (CUDA)
Epoch 10/150 - Train Loss: 0.000006, Val Loss: 0.000006
Epoch 20/150 - Train Loss: 0.000006, Val Loss: 0.000006
Epoch 30/150 - Train Loss: 0.000006, Val Loss: 0.000006
Early stopping at epoch 35

LSTM - Test Metrics:
  MSE: 0.000005
  Sharpe: 2.4071
  Win Rate: 49.64%
  Profit Factor: 1.01

Tempo de treino: 549.7s


## 10. Treinamento: CNN

In [31]:
class CNNNetwork(nn.Module):
    def __init__(self, num_features, seq_length, channels=[64, 128, 256], dropout=0.3):
        super().__init__()
        self.input_proj = nn.Conv1d(num_features, channels[0], kernel_size=1)

        self.conv_blocks = nn.ModuleList()
        in_ch = channels[0]
        for i, out_ch in enumerate(channels):
            self.conv_blocks.append(nn.Sequential(
                nn.Conv1d(in_ch, out_ch, kernel_size=3, padding=1, dilation=2**i),
                nn.BatchNorm1d(out_ch),
                nn.ReLU(),
                nn.Dropout(dropout)
            ))
            in_ch = out_ch

        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels[-1], 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        x = x.transpose(1, 2)  # (batch, features, seq)
        x = self.input_proj(x)
        for block in self.conv_blocks:
            x = block(x)
        x = self.global_pool(x).squeeze(-1)
        return self.fc(x).squeeze(-1)

In [32]:
print("="*60)
print("TREINANDO CNN (CUDA)")
print("="*60)

cnn_model = CNNNetwork(num_features, SEQUENCE_LENGTH, channels=[64, 128, 256]).to(DEVICE)

optimizer = torch.optim.AdamW(cnn_model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=5)
criterion = nn.MSELoss()

best_val_loss = float("inf")
patience_counter = 0

start_time = time.time()
for epoch in range(epochs):
    cnn_model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
        optimizer.zero_grad()
        output = cnn_model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(cnn_model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)

    cnn_model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            output = cnn_model(X_batch)
            val_loss += criterion(output, y_batch).item()
    val_loss /= len(val_loader)

    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_state = cnn_model.state_dict().copy()
    else:
        patience_counter += 1

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        break

cnn_model.load_state_dict(best_state)
cnn_train_time = time.time() - start_time

# Avaliar no TEST set
cnn_model.eval()
with torch.no_grad():
    y_pred_cnn = cnn_model(torch.FloatTensor(X_test_seq).to(DEVICE)).cpu().numpy()

cnn_metrics = evaluate_model(y_test_seq, y_pred_cnn, "CNN")
cnn_metrics["train_time"] = cnn_train_time

all_metrics["cnn"] = cnn_metrics
trained_models["cnn"] = cnn_model

# Salvar
torch.save(cnn_model.state_dict(), str(TRAINED_DIR / "cnn_model.pt"))
print(f"\nTempo de treino: {cnn_train_time:.1f}s")

TREINANDO CNN (CUDA)
Epoch 10/150 - Train Loss: 0.000006, Val Loss: 0.000006
Epoch 20/150 - Train Loss: 0.000006, Val Loss: 0.000006
Epoch 30/150 - Train Loss: 0.000006, Val Loss: 0.000006
Epoch 40/150 - Train Loss: 0.000006, Val Loss: 0.000006
Early stopping at epoch 48

CNN - Test Metrics:
  MSE: 0.000005
  Sharpe: 2.4071
  Win Rate: 49.64%
  Profit Factor: 1.01

Tempo de treino: 429.2s


## 11. Treinamento: D4PG+EVT

In [33]:
# Importar do projeto ou definir inline
# Aqui definimos inline para o notebook ser auto-contido

from collections import deque
from scipy import stats
import random

class EVTRiskModel:
    """Extreme Value Theory para gestão de risco"""
    def __init__(self, threshold_percentile=95.0):
        self.threshold_percentile = threshold_percentile
        self.losses = []
        self.shape = None
        self.scale = None
        self.threshold = None

    def update(self, returns):
        losses = -returns[returns < 0]
        self.losses.extend(losses.tolist())
        if len(self.losses) < 100:
            return
        losses_array = np.array(self.losses)
        self.threshold = np.percentile(losses_array, self.threshold_percentile)
        exceedances = losses_array[losses_array > self.threshold] - self.threshold
        if len(exceedances) >= 10:
            try:
                self.shape, _, self.scale = stats.genpareto.fit(exceedances, floc=0)
            except:
                pass

    def var(self, confidence=0.99):
        if self.shape is None:
            return 0.0
        n = len(self.losses)
        n_exc = sum(1 for l in self.losses if l > self.threshold)
        if n_exc == 0:
            return 0.0
        p = n_exc / n
        q = 1 - confidence
        if self.shape == 0:
            return self.threshold + self.scale * np.log(p / q)
        return self.threshold + (self.scale / self.shape) * ((p / q) ** self.shape - 1)

    def cvar(self, confidence=0.99):
        var = self.var(confidence)
        if self.shape is None or self.shape >= 1:
            return var
        return var / (1 - self.shape) + (self.scale - self.shape * self.threshold) / (1 - self.shape)

In [34]:
class D4PGActor(nn.Module):
    def __init__(self, state_dim, action_dim=1, hidden_dim=256):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim),
            nn.Tanh()
        )

    def forward(self, state):
        return self.net(state)


class D4PGCritic(nn.Module):
    def __init__(self, state_dim, action_dim=1, hidden_dim=256, n_atoms=51):
        super().__init__()
        self.n_atoms = n_atoms
        self.net = nn.Sequential(
            nn.Linear(state_dim + action_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, n_atoms)
        )

    def forward(self, state, action):
        x = torch.cat([state, action], dim=-1)
        return torch.softmax(self.net(x), dim=-1)

In [35]:
class TradingEnvRL:
    """Ambiente de trading para RL - usa APENAS dados de treino!"""
    def __init__(self, data, features, initial_balance=100000, transaction_cost=0.0005):
        self.data = data
        self.features = features
        self.initial_balance = initial_balance
        self.transaction_cost = transaction_cost
        self.reset()

    def reset(self):
        self.balance = self.initial_balance
        self.position = 0.0
        self.step_idx = 0
        self.returns = []
        return self._get_state()

    def _get_state(self):
        market = self.features[self.step_idx]
        portfolio = np.array([
            self.position,
            self.balance / self.initial_balance - 1,
            np.mean(self.returns[-20:]) if self.returns else 0,
            np.std(self.returns[-20:]) if len(self.returns) > 1 else 0
        ])
        return np.concatenate([market, portfolio])

    def step(self, action):
        target_pos = float(np.clip(action[0], -1, 1))
        pos_change = target_pos - self.position
        current_price = self.data[self.step_idx, 3]
        cost = abs(pos_change) * current_price * self.transaction_cost

        self.step_idx += 1
        done = self.step_idx >= len(self.data) - 1

        if not done:
            next_price = self.data[self.step_idx, 3]
            ret = (next_price - current_price) / current_price
            pnl = self.position * ret * self.balance - cost
            self.balance += pnl
            step_ret = pnl / self.initial_balance
            self.returns.append(step_ret)
            self.position = target_pos

            # Reward = Sharpe-like
            if len(self.returns) > 1:
                reward = np.mean(self.returns[-20:]) / (np.std(self.returns[-20:]) + 1e-8)
            else:
                reward = step_ret * 100
        else:
            reward = 0

        return self._get_state() if not done else np.zeros_like(self._get_state()), reward, done

In [36]:
print("="*60)
print("TREINANDO D4PG+EVT (CUDA)")
print("="*60)

# Criar ambiente com dados de TREINO apenas!
state_dim = rl_train_features.shape[1] + 4
env = TradingEnvRL(rl_train_data, rl_train_features)

actor = D4PGActor(state_dim).to(DEVICE)
actor_target = D4PGActor(state_dim).to(DEVICE)
actor_target.load_state_dict(actor.state_dict())

critic = D4PGCritic(state_dim).to(DEVICE)
critic_target = D4PGCritic(state_dim).to(DEVICE)
critic_target.load_state_dict(critic.state_dict())

actor_opt = torch.optim.Adam(actor.parameters(), lr=1e-4)
critic_opt = torch.optim.Adam(critic.parameters(), lr=3e-4)

evt_model = EVTRiskModel()
buffer = deque(maxlen=100000)
batch_size = 256
gamma = 0.99
tau = 0.005
episodes = 200

start_time = time.time()
for ep in range(episodes):
    state = env.reset()
    ep_reward = 0

    while True:
        state_t = torch.FloatTensor(state).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            action = actor(state_t).cpu().numpy()[0]

        # Exploração
        action = action + np.random.normal(0, 0.1, size=action.shape)
        action = np.clip(action, -1, 1)

        next_state, reward, done = env.step(action)
        buffer.append((state, action, reward, next_state, done))
        evt_model.update(np.array([reward]))

        state = next_state
        ep_reward += reward

        # Treinar
        if len(buffer) >= batch_size:
            batch = random.sample(buffer, batch_size)
            states, actions, rewards, next_states, dones = zip(*batch)

            states = torch.FloatTensor(np.array(states)).to(DEVICE)
            actions = torch.FloatTensor(np.array(actions)).to(DEVICE)
            rewards = torch.FloatTensor(rewards).unsqueeze(1).to(DEVICE)
            next_states = torch.FloatTensor(np.array(next_states)).to(DEVICE)
            dones = torch.FloatTensor(dones).unsqueeze(1).to(DEVICE)

            # Critic update
            with torch.no_grad():
                next_actions = actor_target(next_states)
                target_q = critic_target(next_states, next_actions)

            current_q = critic(states, actions)
            critic_loss = nn.MSELoss()(current_q.mean(dim=1), rewards.squeeze() + gamma * (1-dones.squeeze()) * target_q.mean(dim=1))

            critic_opt.zero_grad()
            critic_loss.backward()
            critic_opt.step()

            # Actor update
            actor_loss = -critic(states, actor(states)).mean()

            actor_opt.zero_grad()
            actor_loss.backward()
            actor_opt.step()

            # Soft update
            for p, tp in zip(actor.parameters(), actor_target.parameters()):
                tp.data.copy_(tau * p.data + (1-tau) * tp.data)
            for p, tp in zip(critic.parameters(), critic_target.parameters()):
                tp.data.copy_(tau * p.data + (1-tau) * tp.data)

        if done:
            break

    if (ep + 1) % 20 == 0:
        total_ret = (env.balance - env.initial_balance) / env.initial_balance
        print(f"Episode {ep+1}/{episodes} - Return: {total_ret:.2%}, VaR: {evt_model.var():.4f}")

d4pg_train_time = time.time() - start_time

d4pg_metrics = {
    "var_99": evt_model.var(),
    "cvar_99": evt_model.cvar(),
    "train_time": d4pg_train_time
}

all_metrics["d4pg"] = d4pg_metrics
trained_models["d4pg_actor"] = actor

# Salvar
torch.save(actor.state_dict(), str(TRAINED_DIR / "d4pg_actor.pt"))
print(f"\nTempo de treino: {d4pg_train_time:.1f}s")

TREINANDO D4PG+EVT (CUDA)


KeyboardInterrupt: 

## 12. Treinamento: MARL

In [ ]:
class MARLAgent(nn.Module):
    def __init__(self, state_dim, action_dim=1, hidden_dim=128, message_dim=32, n_agents=5):
        super().__init__()
        self.state_enc = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        self.msg_enc = nn.Sequential(
            nn.Linear(message_dim * (n_agents - 1), hidden_dim // 2),
            nn.ReLU()
        )
        self.policy = nn.Sequential(
            nn.Linear(hidden_dim + hidden_dim // 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim),
            nn.Tanh()
        )
        self.msg_gen = nn.Sequential(
            nn.Linear(hidden_dim, message_dim),
            nn.Tanh()
        )

    def forward(self, state, messages):
        state_emb = self.state_enc(state)
        msg_emb = self.msg_enc(messages.flatten(start_dim=-2))
        combined = torch.cat([state_emb, msg_emb], dim=-1)
        action = self.policy(combined)
        msg_out = self.msg_gen(state_emb)
        return action, msg_out

In [ ]:
print("="*60)
print("TREINANDO MARL (CUDA)")
print("="*60)

n_agents = 5
message_dim = 32
marl_state_dim = rl_train_features.shape[1] + 4

agents = [MARLAgent(marl_state_dim, message_dim=message_dim, n_agents=n_agents).to(DEVICE) for _ in range(n_agents)]
optimizers = [torch.optim.Adam(a.parameters(), lr=3e-4) for a in agents]

# Critic centralizado
critic = nn.Sequential(
    nn.Linear(n_agents * marl_state_dim + n_agents, 256),
    nn.ReLU(),
    nn.Linear(256, 256),
    nn.ReLU(),
    nn.Linear(256, n_agents)
).to(DEVICE)
critic_opt = torch.optim.Adam(critic.parameters(), lr=3e-4)

marl_buffer = deque(maxlen=50000)
episodes = 150

start_time = time.time()
for ep in range(episodes):
    env = TradingEnvRL(rl_train_data, rl_train_features)
    base_state = env.reset()
    states = np.array([base_state + np.random.normal(0, 0.01, size=base_state.shape) for _ in range(n_agents)])
    messages = np.zeros((n_agents, message_dim))

    ep_reward = 0

    while True:
        actions = []
        new_messages = []

        for i, agent in enumerate(agents):
            other_msgs = np.stack([messages[j] for j in range(n_agents) if j != i])
            state_t = torch.FloatTensor(states[i]).unsqueeze(0).to(DEVICE)
            msgs_t = torch.FloatTensor(other_msgs).unsqueeze(0).to(DEVICE)

            with torch.no_grad():
                action, msg = agent(state_t, msgs_t)

            action = action.cpu().numpy()[0] + np.random.normal(0, 0.1, size=(1,))
            action = np.clip(action, -1, 1)
            actions.append(action)
            new_messages.append(msg.cpu().numpy()[0])

        messages = np.array(new_messages)

        # Agregar ações
        agg_action = np.mean(actions, axis=0)
        next_base_state, reward, done = env.step(agg_action)

        next_states = np.array([next_base_state + np.random.normal(0, 0.01, size=next_base_state.shape) for _ in range(n_agents)])
        rewards = np.full(n_agents, reward)

        marl_buffer.append((states.copy(), np.array(actions), rewards, next_states.copy(), done))

        states = next_states
        ep_reward += reward

        # Treinar
        if len(marl_buffer) >= batch_size:
            batch = random.sample(marl_buffer, batch_size)
            b_states, b_actions, b_rewards, b_next_states, b_dones = zip(*batch)

            b_states = torch.FloatTensor(np.array(b_states)).to(DEVICE)
            b_actions = torch.FloatTensor(np.array(b_actions)).to(DEVICE)
            b_rewards = torch.FloatTensor(np.array(b_rewards)).to(DEVICE)

            # Critic update
            critic_input = torch.cat([b_states.view(batch_size, -1), b_actions.view(batch_size, -1)], dim=-1)
            q_values = critic(critic_input)
            critic_loss = nn.MSELoss()(q_values, b_rewards)

            critic_opt.zero_grad()
            critic_loss.backward()
            critic_opt.step()

        if done:
            break

    if (ep + 1) % 20 == 0:
        total_ret = (env.balance - env.initial_balance) / env.initial_balance
        print(f"Episode {ep+1}/{episodes} - Return: {total_ret:.2%}")

marl_train_time = time.time() - start_time

marl_metrics = {
    "n_agents": n_agents,
    "train_time": marl_train_time
}

all_metrics["marl"] = marl_metrics
trained_models["marl_agents"] = agents

# Salvar
for i, agent in enumerate(agents):
    torch.save(agent.state_dict(), str(TRAINED_DIR / f"marl_agent_{i}.pt"))
print(f"\nTempo de treino: {marl_train_time:.1f}s")

## 13. Exportar para ONNX

In [ ]:
print("="*60)
print("EXPORTANDO PARA ONNX")
print("="*60)

# LSTM
lstm_model.eval()
dummy_lstm = torch.randn(1, SEQUENCE_LENGTH, num_features).to(DEVICE)
torch.onnx.export(
    lstm_model, dummy_lstm,
    str(ONNX_DIR / "lstm_model.onnx"),
    input_names=["input"], output_names=["output"],
    dynamic_axes={"input": {0: "batch"}, "output": {0: "batch"}},
    opset_version=17
)
print("LSTM exportado para ONNX")

# CNN
cnn_model.eval()
dummy_cnn = torch.randn(1, SEQUENCE_LENGTH, num_features).to(DEVICE)
torch.onnx.export(
    cnn_model, dummy_cnn,
    str(ONNX_DIR / "cnn_model.onnx"),
    input_names=["input"], output_names=["output"],
    dynamic_axes={"input": {0: "batch"}, "output": {0: "batch"}},
    opset_version=17
)
print("CNN exportado para ONNX")

# D4PG Actor
actor.eval()
dummy_actor = torch.randn(1, state_dim).to(DEVICE)
torch.onnx.export(
    actor, dummy_actor,
    str(ONNX_DIR / "d4pg_actor.onnx"),
    input_names=["state"], output_names=["action"],
    dynamic_axes={"state": {0: "batch"}, "action": {0: "batch"}},
    opset_version=17
)
print("D4PG Actor exportado para ONNX")

# MARL Agents
for i, agent in enumerate(agents):
    agent.eval()
    dummy_state = torch.randn(1, marl_state_dim).to(DEVICE)
    dummy_msgs = torch.randn(1, n_agents-1, message_dim).to(DEVICE)

    class AgentWrapper(nn.Module):
        def __init__(self, a):
            super().__init__()
            self.agent = a
        def forward(self, s, m):
            action, _ = self.agent(s, m)
            return action

    torch.onnx.export(
        AgentWrapper(agent), (dummy_state, dummy_msgs),
        str(ONNX_DIR / f"marl_agent_{i}.onnx"),
        input_names=["state", "messages"], output_names=["action"],
        opset_version=17
    )
print(f"MARL Agents ({n_agents}) exportados para ONNX")

print(f"\nTotal de modelos ONNX: {len(list(ONNX_DIR.glob('*.onnx')))}")

## 14. Resultados Finais

In [ ]:
import matplotlib.pyplot as plt

print("="*60)
print("RESUMO DOS RESULTADOS")
print("="*60)

# Criar DataFrame
metrics_df = pd.DataFrame(all_metrics).T
print("\nMétricas por Modelo:")
print(metrics_df.to_string())

# Ranking
if "sharpe_ratio" in metrics_df.columns:
    print("\n\nRanking por Sharpe Ratio:")
    ranking = metrics_df["sharpe_ratio"].dropna().sort_values(ascending=False)
    for i, (m, s) in enumerate(ranking.items(), 1):
        print(f"{i}. {m}: {s:.4f}")

In [ ]:
# Salvar resultados
results = {
    "metrics": {k: {kk: float(vv) if isinstance(vv, (np.floating, float)) else vv
                    for kk, vv in v.items()} for k, v in all_metrics.items()},
    "config": {
        "symbols": SYMBOLS,
        "days": DAYS,
        "sequence_length": SEQUENCE_LENGTH,
        "test_size": TEST_SIZE,
        "val_size": VAL_SIZE,
        "num_features": num_features
    },
    "anti_leakage_measures": [
        "Temporal split BEFORE any transformation",
        "Scaler fit ONLY on training data",
        "Separate scalers for ML, DL, RL",
        "Features use only past data (rolling windows)",
        "No bfill() that could leak future data",
        "RL trained only on training data"
    ]
}

with open(TRAINED_DIR / "training_results.json", "w") as f:
    json.dump(results, f, indent=2, default=str)

print(f"Resultados salvos em: {TRAINED_DIR / 'training_results.json'}")

In [ ]:
# Listar arquivos gerados
print("\nArquivos Gerados:")
print("\nModelos:")
for f in sorted(TRAINED_DIR.glob("*")):
    if f.is_file():
        print(f"  {f.name} ({f.stat().st_size/1024:.1f} KB)")

print("\nONNX:")
for f in sorted(ONNX_DIR.glob("*.onnx")):
    print(f"  {f.name} ({f.stat().st_size/1024:.1f} KB)")

print("\n" + "="*60)
print("TREINAMENTO COMPLETO - SEM DATA LEAKAGE!")
print("="*60)